In [70]:
#testget_gnd_state_probs_and_approx_ratio.ipynb
from pyaqc.RCModules.aqc_rigetti import get_gnd_state_probs_and_approx_ratio
#ONE AMBULANCE adjacency table
from pyaqc.RCModules.PlatformConversion import print_QUBOdetails, CreateAmbulanceAdjacency,distance #   qc_all_env and rigoct38_env
import numpy as np 
import unicodedata as ud
betagreek =  ud.lookup('GREEK SMALL LETTER BETA')
gammagreek =  ud.lookup('GREEK SMALL LETTER gamma')

################  Set up type of problem to solve ################
n_destinations = 5
n_qubits = n_destinations
HammingWeightOfConstraint =  n_qubits - 1

gridWidth = n_qubits
ConstraintMultiplier = 14              # I have used 14 for 5 qubit with constraints and traditional mixer, 140 for q11, tim 200
remove_constraint=  0   # remove the hard constraints when using an XY mixer
select_qubo_model = 1   # we nearly always use this
################ Create 1) adjacency table of the problem, and 2) of the hard constraints ################
Adjacency_constraint = CreateAmbulanceAdjacency(   
    gridWidth,
    n_qubits = n_qubits, 
    ConstraintMultiplier = ConstraintMultiplier, 
    Adddistance = 0,
    remove_constraint= remove_constraint, 
    HammingWeightOfConstraint= HammingWeightOfConstraint,
    qubo_model=select_qubo_model)
Adjacency = CreateAmbulanceAdjacency(   
    gridWidth,
    n_qubits = n_qubits, 
    ConstraintMultiplier = ConstraintMultiplier, 
    Adddistance = 1,
    remove_constraint= remove_constraint, 
    HammingWeightOfConstraint= HammingWeightOfConstraint,
    qubo_model=select_qubo_model)
################ ... and print ################
if select_qubo_model:print('Qubo ', end='')
else: print('Ising ', end='')
print_QUBOdetails(Adjacency,n_qubits, 'Ambulance problem')
n_ambulance=1
Ansatz_type = {'n_qubits':n_qubits, 'p':0,'Mixer':0,'prog_init':0, 'ConstraintMultiplier':0, 'remove_constraint':0, 'Optimizer_maxiter': 0}
Ansatz_type['ConstraintMultiplier'] = ConstraintMultiplier
Ansatz_type['remove_constraint'] = remove_constraint
############################# convert Adjacency to Pauli sum of observeable edges and nodes #############################
from pyaqc.RCModules.aqc_rigetti import create_ring_mixer, Adjacency_qubo_to_Regetti,Adjacency_Ising_to_Regetti,Dicke_state
ListPauli_termsMy,SumPauli_termsMy  = Adjacency_qubo_to_Regetti( Adjacency, n_qubits)

print(SumPauli_termsMy)

from pyaqc.RCModules.aqc_rigetti import get_gnd_state_probs_and_approx_ratio,ansatz_init_XYMixer,ansatz_prog_init,ansatz_EV_init
from pyaqc.RCModules.aqc_rigetti import create_ring_mixer,create_local_ring_mixer,Dicke_state , Dicke_state_local
from pyquil.paulis import *  #  exponential_map, exponentiate_commuting_pauli_sum
from pyquil.api import *
from pyquil.gates import *

def get_mixer(n_qubits,remove_constraint,n_ambulance ):
    """
    Return Rigetti observables that can be exponentiated, and a description 'Ansatz_type'
    type: Pauli Observables
    """
    MyMixerHam = []
    MyMixerHam.append(0)                # Perhaps required by QAOA()
    if not remove_constraint:           # when the extra hard constraints are in place use an X mixer, otherwise an XY mixer
        for n in range(n_qubits):       
            MyMixerHam[0] += 1*sX(n) 
        Ansatz_type['Mixer'] ='TRADITIONAL /  Farhi  mixer'
    else:
        XYmixer = create_ring_mixer(n_qubits)
        MyMixerHam[0] = XYmixer         #XY
        Ansatz_type['Mixer'] ='XY Ring mixer'
        if n_ambulance==2:
            MyMixerHam, prog_init_three_XYMixers = three_XYMixers_Dicke_states(n_qubits, n_destinations) 
            Ansatz_type['prog_init'] = 'Three Dicke states'
            Ansatz_type['Mixer'] ='Three XY Ring mixers' 
    
    if n_ambulance==2:
        labels= ['Destination XYmixer','Start0 XYmixer','Start1 XYmixer']
        for n_mixer, mixer in enumerate(MyMixerHam):
            print(labels[n_mixer])
            print(mixer)
    else: 
        print(Ansatz_type['Mixer'])
        print(MyMixerHam[0])        
        
    return MyMixerHam, Ansatz_type
def get_prog_init(n_qubits,remove_constraint,n_ambulance ):
    """
    Return the Quantum circuit of the initial state (eg Hamamard for Xmixer)
    """
    prog_init = Program()

    if not remove_constraint:
        #A) 
        print('Hadamard initial state.   ', n_qubits, ' = n_qubits')
        for n in range(n_qubits):
            prog_init += H(n)
    else:
        if n_ambulance==2:
            #B)
            #MyMixerHam, prog_init_three_XYMixers = three_XYMixers_Dicke_states(n_qubits, n_destinations)  
            prog_init = prog_init_three_XYMixers       # this is just a reminder that the calculation was made earlier in the 'Mixer used' cell
            print('The initial state is made of three separate Dicke_state s')
        else:
            #C)
            print('Dicke_state initial state',HammingWeightOfConstraint, ' = HW')
            prog_init = Dicke_state(n_qubits, HammingWeightOfConstraint)

    return prog_init

MyMixerHam, Ansatz_type = get_mixer(n_qubits,remove_constraint,n_ambulance )
prog_init = get_prog_init(n_qubits,remove_constraint,n_ambulance )



Qubo ADJACENCY MATRIX Ambulance problem  

Qubit  q0   q1   q2   q3   q4 

  q0  -98   27   24   19   12 

  q1       -98   27   24   19 

  q2            -98   27   24 

  q3                 -98   27 

  q4                      -98 

(-187.5+0j)*I + (28.5+0j)*Z0 + (6.75+0j)*Z0*Z1 + (24.75+0j)*Z1 + (6+0j)*Z0*Z2 + (23.5+0j)*Z2 + (4.75+0j)*Z0*Z3 + (24.75+0j)*Z3 + (3+0j)*Z0*Z4 + (28.5+0j)*Z4 + (6.75+0j)*Z1*Z2 + (6+0j)*Z1*Z3 + (4.75+0j)*Z1*Z4 + (6.75+0j)*Z2*Z3 + (6+0j)*Z2*Z4 + (6.75+0j)*Z3*Z4
TRADITIONAL /  Farhi  mixer
(1+0j)*X0 + (1+0j)*X1 + (1+0j)*X2 + (1+0j)*X3 + (1+0j)*X4
Hadamard initial state.    5  = n_qubits


In [79]:
# code plan: find bg for a q5 1 A problem
q5p1TradXmHqq = [6.898, -0.022] # nm -252.8 #11.1% Energy_max =-20 , Energy_min = -40 when constraint are complied with and have zero energy contribution to the cost function
opt_betagamma = q5p1TradXmHqq = [6.898, -0.022] # nm -252.8 #11.1%
p = len(opt_betagamma)//2
state_feasible = '11011'
ansatz_prog = ansatz_prog_init(prog_init,   ListPauli_termsMy, n_qubits, n_destinations,   p,MyMixerHam)
if 0:
    prob_gnd_state, approx_ratio,prob_feasible_state = get_gnd_state_probs_and_approx_ratio(opt_betagamma,ansatz_prog,SumPauli_termsMy,n_qubits,Adjacency_constraint=Adjacency_constraint, state_feasible=state_feasible, prt_details=1)
    print(prob_gnd_state, approx_ratio,prob_feasible_state)
from pyaqc.RCModules.QAOARC import decimal_state_to_binary_reversed, Energy_of_binary_state
from pyaqc.RCModules.aqc_rigetti import min_energy
def get_approx_ratio_init(ansatz_prog,prog_init,ListPauli_termsMy,SumPauli_termsMy,n_qubits,p,Adjacency_constraint=None,  prt_details=1,state_feasible=None ):
    """
    Initialise knowing that         
        1) Each call will be for the same ising problem. Hence, it will use the same E_min_fease,  E_max_fease, and min_energy_constraint
        2) each call of 'get_gnd_state_probs_and_approx_ratio' will use a different betagamma, and the same p if p is supplied, otherwise a p based on opt_betagamma

    """
    
    ### Calculate min_energy_constraint, then E_min_feas and E_max_fease  by a 2^n search
    if not Adjacency_constraint==None: 
        ' get  Minimum energy of the Adjacency by considering all 2^n_qubits states. OR if state_feasible is supplied the adjacenys energy in that state  type:float'
        if isinstance(state_feasible,str):
            state_feasible=state_feasible.replace(' ','')
        min_energy_constraint = min_energy( Adjacency_constraint, n_qubits, state_feasible=state_feasible)#state_feasible
        ListPauli_terms_constr,SumPauli_termsMy_constr  = Adjacency_qubo_to_Regetti( Adjacency_constraint, n_qubits)

        def energy_min_max_feasible():
            seen_first_compliant_state = 0
            E_min_feas = Energy_of_binary_state( 0, SumPauli_termsMy,n_qubits)-min_energy_constraint
            for n in range(2**n_qubits):
                E_feas_of_state = Energy_of_binary_state( n, SumPauli_termsMy,n_qubits)-min_energy_constraint
                E_min_feas = min(E_min_feas,E_feas_of_state)
                if Energy_of_binary_state( n, SumPauli_termsMy_constr, n_qubits) == min_energy_constraint:
                    if seen_first_compliant_state==0:
                        E_max_feas = E_feas_of_state
                    E_max_feas = max(E_max_feas,E_feas_of_state)
                    seen_first_compliant_state = 1
            return E_min_feas, E_max_feas 
        E_min_feas, E_max_feas = energy_min_max_feasible()
        print(E_min_feas, E_max_feas,min_energy_constraint,'E_min_feas, E_max_feas, min_energy_constraint')
    
    def get_gnd_state_probs_and_approx_ratio(opt_betagamma,use_new_ansatz_prog=False,  prt_details=1):  
        """
        if use_new_ansatz_prog=True, the ansatz_prog is re initiated each call of get_gnd_state_probs_and_approx_ratio, to reflect the p implied by opt_betagamma (eg when creating the approx ratio for many different p s of the same problem)
        if use_new_ansatz_prog=false, execution time is saved by using the ansatz_prog that was supplied by get_approx_ratio_init (eg when looking for betagamma minimum)
        """
        prob_gnd_state = 0
        prob_feasible_state = 0
        EV_feasible = 0 
        n_feasible = 0
        approx_ratio = -1
        if use_new_ansatz_prog:
            p = len(opt_betagamma)//2
            ansatz_prog_flexible_p = ansatz_prog_init(prog_init, ListPauli_termsMy, n_qubits, n_destinations,   p,MyMixerHam)
            solution_ansatz = ansatz_prog_flexible_p(opt_betagamma)
        else:
            solution_ansatz = ansatz_prog(opt_betagamma)
        
        # dovetail rest from existing fucntion
        probsAbsolute  = WavefunctionSimulator().wavefunction(solution_ansatz).get_outcome_probs()
        opt_probs_absolute = [x for x in probsAbsolute.values()]
        for n in range(2**n_qubits):
        
            if opt_probs_absolute[n] > 0.00001 :    #|qn...q0>
                # calculation of energy, using n as the state, NOT recalculating the state by building its quantum circuit eg Program(X(0),I(1))
                Expectation_of_state = Energy_of_binary_state( n, SumPauli_termsMy,n_qubits) - min_energy_constraint   


            if Expectation_of_state <= E_min_feas:   
                
                psi_opt, state_string = decimal_state_to_binary_reversed(n,n_qubits)
                prob_gnd_state += opt_probs_absolute[n]

            if not Adjacency_constraint==None:
                # If the Adjacency_constraint has an energy that is the minimium possible, the constraints have been met..
                #...then the energy calculated without any constraint energy is averaged
                if Energy_of_binary_state( n, SumPauli_termsMy_constr, n_qubits) == min_energy_constraint:
                    
                    EV_feasible += opt_probs_absolute[n] * (Expectation_of_state )
                    prob_feasible_state +=opt_probs_absolute[n]
                    n_feasible +=1
        approx_ratio = (EV_feasible-E_max_feas)/(E_min_feas-E_max_feas)
        return prob_gnd_state, approx_ratio,prob_feasible_state
    return get_gnd_state_probs_and_approx_ratio

#UseBetaGamma = { 'Betas':3, 'Gammas':3}
ansatz_prog = ansatz_prog_init(prog_init, ListPauli_termsMy, n_qubits, n_destinations,   p, MyMixerHam)  
state_feasible =None# '11011'
get_gnd_state_probs_and_approx_ratio = get_approx_ratio_init(ansatz_prog,prog_init,ListPauli_termsMy,SumPauli_termsMy,n_qubits,p,Adjacency_constraint=Adjacency_constraint,  prt_details=1,state_feasible=state_feasible)
get_gnd_state_probs_and_approx_ratio(opt_betagamma,use_new_ansatz_prog=0)

-40.0 -20.0 -224.0 E_min_feas, E_max_feas, min_energy_constraint


(0.9989132859304007, -0.30469836247037974, 0.4388586992135832)